In [13]:
#https://pennylane.ai/codebook/learning-pathsclea

In [14]:
import numpy as np

# IQC

## All about qubits

A common misconception is that a qubit in a superposition of two states is in both states at the same time. This is false; the qubit is only ever in one state. It's just that sometimes, that state may be a linear combination of the basis states.

### Normalization of quantum state

In [15]:
# Here are the vector representations of |0> and |1>, for convenience
ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])


def normalize_state(alpha, beta):
    """Compute a normalized quantum state given arbitrary amplitudes.

    Args:
        alpha (complex): The amplitude associated with the |0> state.
        beta (complex): The amplitude associated with the |1> state.

    Returns:
        np.array[complex]: A vector (numpy array) with 2 elements that represents
        a normalized quantum state.
    """
    param_k = 1/np.sqrt(np.abs(alpha)**2 + np.abs(beta)**2)
    alpha_ = param_k * alpha
    beta_ = param_k * beta

    return np.array([alpha_, beta_])
    



### Inner product and orthonormal basis

In [16]:
def inner_product(state_1, state_2):
    """Compute the inner product between two states.

    Args:
        state_1 (np.array[complex]): A normalized quantum state vector
        state_2 (np.array[complex]): A second normalized quantum state vector

    Returns:
        complex: The value of the inner product <state_1 | state_2>.
    """

    bra_state_1 = np.conjugate(state_1)

    return np.inner(bra_state_1, state_2)


# Test your results with this code
ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])

print(f"<0|0> = {inner_product(ket_0, ket_0)}")
print(f"<0|1> = {inner_product(ket_0, ket_1)}")
print(f"<1|0> = {inner_product(ket_1, ket_0)}")
print(f"<1|1> = {inner_product(ket_1, ket_1)}")


<0|0> = 1
<0|1> = 0
<1|0> = 0
<1|1> = 1


### Sampling measurement outcomes

In [21]:
def measure_state(state, num_meas):
    """Simulate a quantum measurement process.

    Args:
        state (np.array[complex]): A normalized qubit state vector.
        num_meas (int): The number of measurements to take

    Returns:
        np.array[int]: A set of num_meas samples, 0 or x1, chosen according to the probability
        distribution defined by the input state.
    """

    prob = np.abs(state)**2

    return np.random.choice([0, 1], size = num_meas, p = prob)


### Applying a quantum operation

In [22]:
U = np.array([[1, 1], [1, -1]]) / np.sqrt(2)


def apply_u(state):
    """Apply a quantum operation.

    Args:
        state (np.array[complex]): A normalized quantum state vector.

    Returns:
        np.array[complex]: The output state after applying U.
    """
    
    return np.dot(U, state)


### A simple quantum algorithm